In [1]:
import polars as pl
import pyodbc

In [42]:
base_vigente = pl.read_excel("../data/external/Base Vigente Diciembre 2024.xlsx",
                   schema_overrides={"tpro_clave_original": pl.String, 
                                     "tpro_clave": pl.String})

In [43]:
# Agregar archivos .accdb con ´sudo nano /etc/odbc.ini´
dsn = "Catalogos_Dic24"
user = ""
password = ""

conn = pyodbc.connect(f"DSN={dsn};UID={user};PWD={password}")
cursor = conn.cursor()

banco = pl.read_database(query='SELECT * FROM [BANCO]', connection=conn)
agrupamiento = pl.read_database(query='SELECT * FROM [AGRUPAMIENTO]', connection=conn)
programa = pl.read_database(query='SELECT * FROM [PROGRAMA]', connection=conn, schema_overrides={'PROGRAMA_ID': pl.Int128})
garantia = pl.read_database(query='SELECT * FROM [GARANTIA]', connection=conn)
tipo_credito = pl.read_database(query='SELECT * FROM [TIPO_CREDITO]', connection=conn, schema_overrides={'Tipo_Credito_ID': pl.Int32})


conn.close()


In [11]:
garantia

Producto,Programa,Programa_SIAG
str,str,str
"""EMPRESA MEDIANA""","""Mediana""","""EMPRESA MEDIANA"""
"""GARANTIA EMERGENTES""","""Emergencias""","""EMERGENTE"""
"""GARANTIA EMPRESARIAL""","""Empresarial""","""EMPRESARIAL"""
"""GARANTIA ESPECIAL""","""Especial""","""ESPECIAL"""
"""GARANTIA FIANZA""","""Fianzas""","""FIANZAS"""
"""GARANTIA MICROCREDITO""","""Microcrédito""","""MICROCREDITO"""
"""GARANTIA SECTORIAL""","""Sectorial""","""SECTORIAL"""
"""GARANTIA SELECTIVA""","""Selectiva""","""SELECTIVAS"""
"""GARANTIA SUBASTA""","""Subasta""","""SUBASTA"""


In [44]:
mes = (
    base_vigente
    .join(tipo_credito.select([
        "Clave (Garantias)",
        "Tipo_Credito_ID"
    ]), left_on= "ticr_clave", right_on="Clave (Garantias)", how='left')

)

In [ ]:
(
    mes
    .join()
)

rfc,tipgar_clave,fecha_valor,plazo,inter_clave,tpro_clave,clave_credito,fecha_registro_garantia,ticr_clave,plazo_dias,tpro_clave_original,descripcion portafolio,SALDO GARANTIZADO,SALDO INDUCIDO,FISO,IF,Tipo_Credito_ID
str,i64,date,i64,i64,i64,str,date,i64,i64,i16,str,f64,f64,str,str,i32
"""LOGA9105191I8""",4,2015-10-08,127,73,1795,"""00740997009654479734""",2015-11-25,14,23,null,"""SECTORIAL""",5680.016,7100.02,"""1148""","""BBVA""",25
"""HEMF670802D20""",1,2015-01-22,120,626,3426,"""00000017784500215001""",2016-07-15,53,9,null,"""EMPRESARIAL""",4375.39,8750.78,"""1148""","""BANCA AFIRME""",63
"""MAML941122JR7""",4,2015-10-08,125,73,1795,"""00740997009837043457""",2015-11-25,14,23,null,"""SECTORIAL""",7436.48,9295.6,"""1148""","""BBVA""",25
"""AAEL800413K80""",4,2014-10-03,120,304,2542,"""22565""",2014-10-13,42,0,null,"""EMERGENTE""",12369.44,12369.44,"""1148""","""BAJIO""",52
"""SARA930623""",4,2015-03-06,120,17,2326,"""14506257""",2015-08-31,19,0,null,"""SECTORIAL""",10433.328,13041.66,"""1148""","""HSBC MEXICO""",30
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""APC130228HK3""",5,2024-09-17,36,12056,21452,"""902233_PASIVOS""",2024-10-24,2,0,null,"""FEFA""",2.1946e7,2.1946e7,"""1148""","""FEFA""",2
"""FAS090113NW7""",5,2024-09-17,36,12056,21452,"""902240_LIQUIDEZ""",2024-10-24,3,0,null,"""FEFA""",3.03e7,3.03e7,"""1148""","""FEFA""",3
"""AFI080212N57""",5,2024-11-15,36,12056,21452,"""918344_LIQUIDEZ""",2024-12-31,3,0,null,"""FEFA""",1.5e7,1.5e7,"""1148""","""FEFA""",3


In [41]:
mes

rfc,tipgar_clave,fecha_valor,plazo,inter_clave,tpro_clave,clave_credito,fecha_registro_garantia,ticr_clave,plazo_dias,tpro_clave_original,descripcion portafolio,SALDO GARANTIZADO,SALDO INDUCIDO,FISO,IF,Tipo_Credito_ID
str,i64,date,i64,i64,i64,str,date,i64,i64,i16,str,f64,f64,str,str,i32
"""LOGA9105191I8""",4,2015-10-08,127,73,1795,"""00740997009654479734""",2015-11-25,14,23,null,"""SECTORIAL""",5680.016,7100.02,"""1148""","""BBVA""",25
"""HEMF670802D20""",1,2015-01-22,120,626,3426,"""00000017784500215001""",2016-07-15,53,9,null,"""EMPRESARIAL""",4375.39,8750.78,"""1148""","""BANCA AFIRME""",63
"""MAML941122JR7""",4,2015-10-08,125,73,1795,"""00740997009837043457""",2015-11-25,14,23,null,"""SECTORIAL""",7436.48,9295.6,"""1148""","""BBVA""",25
"""AAEL800413K80""",4,2014-10-03,120,304,2542,"""22565""",2014-10-13,42,0,null,"""EMERGENTE""",12369.44,12369.44,"""1148""","""BAJIO""",52
"""SARA930623""",4,2015-03-06,120,17,2326,"""14506257""",2015-08-31,19,0,null,"""SECTORIAL""",10433.328,13041.66,"""1148""","""HSBC MEXICO""",30
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""APC130228HK3""",5,2024-09-17,36,12056,21452,"""902233_PASIVOS""",2024-10-24,2,0,null,"""FEFA""",2.1946e7,2.1946e7,"""1148""","""FEFA""",2
"""FAS090113NW7""",5,2024-09-17,36,12056,21452,"""902240_LIQUIDEZ""",2024-10-24,3,0,null,"""FEFA""",3.03e7,3.03e7,"""1148""","""FEFA""",3
"""AFI080212N57""",5,2024-11-15,36,12056,21452,"""918344_LIQUIDEZ""",2024-12-31,3,0,null,"""FEFA""",1.5e7,1.5e7,"""1148""","""FEFA""",3


In [50]:
saldos_mes = (mes
    #.filter(~pl.col("descripcion portafolio").is_in(["SUBASTA", "SELECTIVA", "SELECTIVAS"]))
    .join(banco, left_on="inter_clave", right_on="Intermediario_ID_SIAG", how="left")
    .with_columns(
        pl.when(pl.col("tpro_clave").str.len_chars() <= 3)
        .then(3000 + pl.col("tpro_clave").cast(pl.Int64))
        .otherwise(30000 + pl.col("tpro_clave").cast(pl.Int64))
        .alias("Programa_Id"),
        pl.when(pl.col("tpro_clave_original").str.len_chars() <= 3)
        .then(3000 + pl.col("tpro_clave_original").cast(pl.Int64))
        .otherwise(30000 + pl.col("tpro_clave_original").cast(pl.Int64))
        .fill_null(3999)
        .alias("Programa_Original")
    )
    .join(programa, left_on="Programa_Id", right_on="PROGRAMA_ID", how="left")
    .with_columns(
        pl.when(pl.col("ESQUEMA") == "1P").then(pl.lit("1P")).otherwise(pl.lit("PP")).alias("ESQUEMA")
    )
    .join(garantia, left_on="descripcion portafolio", right_on="Programa_SIAG", how="left")
    .rename({"Producto": "Taxonomia"})
    .join(agrupamiento, left_on="AGRUPAMIENTO_ID", right_on="AGRUPAMIENTO_ID", how="left")
)

saldos_mes

rfc,tipgar_clave,fecha_valor,plazo,inter_clave,tpro_clave,clave_credito,fecha_registro_garantia,ticr_clave,plazo_dias,tpro_clave_original,descripcion portafolio,SALDO GARANTIZADO,SALDO INDUCIDO,FISO,IF,Tipo_Credito_ID,Intermediario_Id,Razón Social ,Banco,Intermediario_SIAG,Programa_Id,Programa_Original,PROGRAMA,AGRUPAMIENTO_ID,ESQUEMA,SUBESQUEMA,FONDOS_CONTRAGARANTIA,Taxonomia,Programa,AGRUPAMIENTO
str,i64,date,i64,i64,str,str,date,i64,i64,str,str,f64,f64,str,str,i32,str,str,str,str,i64,i64,str,f64,str,str,str,str,str,str
"""LOGA9105191I8""",4,2015-10-08,127,73,"""1795""","""00740997009654479734""",2015-11-25,14,23,null,"""SECTORIAL""",5680.016,7100.02,"""1148""","""BBVA""",25,"""10040012""","""BBV BANCOMER""","""Bancomer""","""BBVA MEXICO""",31795,3999,"""BBVA UNIV TECNOLOGICA 80% 1.2%""",1315.0,"""PP""","""Tradicional""","""CF""","""GARANTIA SECTORIAL""","""Sectorial""","""Educacion"""
"""HEMF670802D20""",1,2015-01-22,120,626,"""3426""","""00000017784500215001""",2016-07-15,53,9,null,"""EMPRESARIAL""",4375.39,8750.78,"""1148""","""BANCA AFIRME""",63,"""10040062""","""BANCA AFIRME""","""Afirme""","""BANCA AFIRME""",33426,3999,"""BANCA AFIRME MIPYME (CREDITO C…",1990.0,"""PP""","""Tradicional""","""CF""","""GARANTIA EMPRESARIAL""","""Empresarial""","""Credito Comercial Micro"""
"""MAML941122JR7""",4,2015-10-08,125,73,"""1795""","""00740997009837043457""",2015-11-25,14,23,null,"""SECTORIAL""",7436.48,9295.6,"""1148""","""BBVA""",25,"""10040012""","""BBV BANCOMER""","""Bancomer""","""BBVA MEXICO""",31795,3999,"""BBVA UNIV TECNOLOGICA 80% 1.2%""",1315.0,"""PP""","""Tradicional""","""CF""","""GARANTIA SECTORIAL""","""Sectorial""","""Educacion"""
"""AAEL800413K80""",4,2014-10-03,120,304,"""2542""","""22565""",2014-10-13,42,0,null,"""EMERGENTE""",12369.44,12369.44,"""1148""","""BAJIO""",52,"""10040030""","""BANCO DEL BAJIO""","""Bajío""","""BANCO DEL BAJIO""",32542,3999,"""BAJIO EMERGENTE REAC. ECONOC M…",1420.0,"""PP""","""Tradicional""","""CF""","""GARANTIA EMERGENTES""","""Emergencias""","""Emergente Michoacan 2014"""
"""SARA930623""",4,2015-03-06,120,17,"""2326""","""14506257""",2015-08-31,19,0,null,"""SECTORIAL""",10433.328,13041.66,"""1148""","""HSBC MEXICO""",30,"""10000171""","""HSBC MEXICO""","""HSBC""","""HSBC MEXICO""",32326,3999,"""HSBC UNIV TECNOLOGICA DE MEXIC…",1315.0,"""PP""","""Tradicional""","""CF""","""GARANTIA SECTORIAL""","""Sectorial""","""Educacion"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""APC130228HK3""",5,2024-09-17,36,12056,"""21452""","""902233_PASIVOS""",2024-10-24,2,0,null,"""FEFA""",2.1946e7,2.1946e7,"""1148""","""FEFA""",2,"""467857270""","""FONDO ESPECIAL PARA FINANCIAMI…","""FEFA""","""FONDO ESPECIAL PARA FINANCIAMI…",51452,3999,null,null,"""PP""",null,null,"""GARANTIAS FEFA""","""FEFA""",null
"""FAS090113NW7""",5,2024-09-17,36,12056,"""21452""","""902240_LIQUIDEZ""",2024-10-24,3,0,null,"""FEFA""",3.03e7,3.03e7,"""1148""","""FEFA""",3,"""467857270""","""FONDO ESPECIAL PARA FINANCIAMI…","""FEFA""","""FONDO ESPECIAL PARA FINANCIAMI…",51452,3999,null,null,"""PP""",null,null,"""GARANTIAS FEFA""","""FEFA""",null
"""AFI080212N57""",5,2024-11-15,36,12056,"""21452""","""918344_LIQUIDEZ""",2024-12-31,3,0,null,"""FEFA""",1.5e7,1.5e7,"""1148""","""FEFA""",3,"""467857270""","""FONDO ESPECIAL PARA FINANCIAMI…","""FEFA""","""FONDO ESPECIAL PARA FINANCIAMI…",51452,3999,null,null,"""PP""",null,null,"""GARANTIAS FEFA""","""FEFA""",null


In [49]:
garantia

Producto,Programa,Programa_SIAG
str,str,str
"""EMPRESA MEDIANA""","""Mediana""","""EMPRESA MEDIANA"""
"""GARANTIA EMERGENTES""","""Emergencias""","""EMERGENTE"""
"""GARANTIA EMPRESARIAL""","""Empresarial""","""EMPRESARIAL"""
"""GARANTIA ESPECIAL""","""Especial""","""ESPECIAL"""
"""GARANTIA FIANZA""","""Fianzas""","""FIANZAS"""
"""GARANTIA MICROCREDITO""","""Microcrédito""","""MICROCREDITO"""
"""GARANTIA SECTORIAL""","""Sectorial""","""SECTORIAL"""
"""GARANTIA SELECTIVA""","""Selectiva""","""SELECTIVAS"""
"""GARANTIA SUBASTA""","""Subasta""","""SUBASTA"""
